In [11]:
%reset -f
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_curve, auc
from torch.utils.data import random_split, DataLoader


from customDatasets.audioDataset import AudioDataset


In [12]:
# free gpu
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [13]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, encoding_dim),
            nn.LeakyReLU(0.2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 512),
            nn.ReLU(),
            nn.Linear(512, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [14]:
# test for deciding the mels parameters
from utils.audioUtils import AudioUtil
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
audio_file = "./data/train/normal_id_00_00000000.wav"

aud = AudioUtil.open(audio_file)
sig, sr = aud
mel = MelSpectrogram(sr, n_fft=1000, hop_length=501, n_mels=128)
spec = mel(sig)
ampl = AmplitudeToDB(top_db=80)
spec = ampl(spec)



print(spec.shape)

torch.Size([1, 128, 320])


In [15]:
def train_model(model, train_dl, val_dl, test_dl, criterion, optimizer, device, epochs=5):
    lr_scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    for epoch in range(epochs):
        train_losses = []
        val_losses = []

        for inputs, labels in train_dl:
            model.train()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, inputs.view(inputs.size(0), -1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        lr_scheduler.step()
        print(f'Epoch[{epoch + 1}/{epochs}], Train loss: {np.average(train_losses): .4f}')
        
        
        for inputs, labels in val_dl:
            model.eval()
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, inputs.view(inputs.size(0), -1))
                val_losses.append(loss.item())
        print(f'Epoch[{epoch + 1}/{epochs}], Val loss: {np.average(val_losses): .4f}')
 
        scores = []
        full_labels = []
        for inputs, labels in test_dl:
            model.eval()
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                mse = torch.sum((outputs - inputs.view(inputs.size(0), -1)) ** 2, dim=1) / outputs.shape[1]
                scores.append(mse)
                full_labels.append(labels)
        
        full_labels = torch.cat([label for label in full_labels])
        scores = torch.cat([score for score in scores])
        fpr, tpr, _ = roc_curve(full_labels.cpu().detach(), scores.cpu().detach(), pos_label=0)
        roc_auc = auc(fpr, tpr)
        print(roc_auc)

In [16]:
def set_seed(seed = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

CONFIG = {
    "seed": 42,
    "epochs": 20,
    "num_classes": 2,
    "learning_rate": 0.01,
    "train_batch_size": 32,
    "val_batch_size": 16,
    "test_batch_size": 128,
    "criterion": nn.MSELoss(),
    "device":
        torch.device(
            "cuda:0" if torch.cuda.is_available()
            else "mps" if torch.backends.mps.is_available()
            else "cpu"
        )
}

set_seed(CONFIG['seed'])

data_path = "./data/train/"
data_path_test = "./data/test/"


meta_train_df = pd.read_csv("./data/train.csv")
meta_test_df = pd.read_csv("./data/test.csv")

train_df = meta_train_df[['filename', 'is_normal', 'machine_id']]
train_dataset = AudioDataset(train_df, data_path)
test_df = meta_test_df[['filename', 'is_normal', 'machine_id']]
test_dataset = AudioDataset(test_df, data_path_test)

num_items = len(train_dataset)
num_train = int(0.8 * num_items)
num_val = num_items-num_train

train_ds, val_ds = random_split(train_dataset, [num_train, num_val])
test_ds = test_dataset


train_dl = DataLoader(train_ds, batch_size=CONFIG['train_batch_size'], shuffle=True)
val_dl = DataLoader(val_ds, batch_size=CONFIG['val_batch_size'], shuffle=False)
test_dl = DataLoader(test_ds, batch_size=CONFIG["test_batch_size"], shuffle=False)

In [17]:
input_size = 128 * 320
model = Autoencoder(input_size, encoding_dim=128)
model = model.to(CONFIG["device"])
optimizer = optim.Adam(model.parameters(), lr=CONFIG["learning_rate"])

In [18]:
# compute metrics
inputs_cat=[]
for inputs, labels in train_dl:
    inputs_cat.append(inputs)
inputs_cat = torch.cat([input for input in inputs_cat])
print(inputs_cat.shape)

torch.Size([1896, 1, 320, 128])


In [19]:
# compute the min and max value for each frequency of the batch_sizexchannelxtimexfrequecy
min = torch.min(inputs_cat, dim=0).values
max = torch.max(inputs_cat, dim=0).values
print(max.shape)
print(min.shape)
train_dataset.min = min
train_dataset.max = max
test_dataset.min = min
test_dataset.max = max

torch.Size([1, 320, 128])
torch.Size([1, 320, 128])


In [20]:
train_model(model, train_dl, val_dl, test_dl, CONFIG["criterion"], optimizer, CONFIG["device"], CONFIG["epochs"])

Epoch[1/20], Train loss:  0.2260
Epoch[1/20], Val loss:  0.2291
0.772808988764045
Epoch[2/20], Train loss:  0.2304
Epoch[2/20], Val loss:  0.2332
0.7762588431127757
Epoch[3/20], Train loss:  0.2325
Epoch[3/20], Val loss:  0.2327
0.7755909280066584
Epoch[4/20], Train loss:  0.2310
Epoch[4/20], Val loss:  0.2289
0.7816978776529339
Epoch[5/20], Train loss:  0.2289
Epoch[5/20], Val loss:  0.2308
0.7713774448605909
Epoch[6/20], Train loss:  0.2302
Epoch[6/20], Val loss:  0.2305
0.7739741989180192
Epoch[7/20], Train loss:  0.2298
Epoch[7/20], Val loss:  0.2305
0.7755264253017062
Epoch[8/20], Train loss:  0.2299
Epoch[8/20], Val loss:  0.2302
0.7769704535996671
Epoch[9/20], Train loss:  0.2297
Epoch[9/20], Val loss:  0.2299
0.7739325842696629
Epoch[10/20], Train loss:  0.2300
Epoch[10/20], Val loss:  0.2303
0.7760258010819808
Epoch[11/20], Train loss:  0.2302
Epoch[11/20], Val loss:  0.2310
0.7752933832709114
Epoch[12/20], Train loss:  0.2296
Epoch[12/20], Val loss:  0.2301
0.7672784019975032